In [658]:
import pandas as pd
from config import PATH_TO_EXCEL_FILE, EXELL_SHEET_NAME_1, EXELL_SHEET_NAME_2, EXELL_SHEET_NAME_3, DRIVER, SERVER, DATABASE_1, DATABASE_2, START_DATE, END_DATE, QUERY_1, QUERY_2
from oee_functions import load_excel_sheet, load_from_db_sqlalchemy



Загрузка всех датафемов из:
1 ексель файлам
2 из баз данных

In [659]:
df_operations = load_excel_sheet(PATH_TO_EXCEL_FILE, EXELL_SHEET_NAME_1)
df_operations.head()

,Category of Losses OEE,Type of work,Events
0,Planned_downtime,Daily preventive MAINT,WT_PPR
1,Planned_downtime,Peventive MAINT TO1 (Rig),RP_MAINTENANCE1
2,Planned_downtime,Peventive MAINT TO2 (Rig),RP_MAINTENANCE2
3,Planned_downtime,Peventive MAINT TO3 (Rig),RP_MAINTENANCE3
4,Planned_downtime,Logging waiting time,LG_PHASE_1


In [660]:
df_rig_name = load_excel_sheet(PATH_TO_EXCEL_FILE, EXELL_SHEET_NAME_2)
df_rig_name.head()

,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation
0,BurGeoProekt,PRAKLA_01-BGP,NaN,Direct
1,BurGeoProekt,PRAKLA_02-BGP,NaN,Direct
2,BurGeoProekt,PRAKLA_03-BGP,NaN,Direct
3,BurGeoProekt,PRAKLA_04-BGP,NaN,Direct
4,BurGeoProekt,ZIF1200_01-BGP,ZIF 1200 №1 Бургеопроект,Direct


In [661]:
df_drill_norms = load_excel_sheet(PATH_TO_EXCEL_FILE, EXELL_SHEET_NAME_3)
df_drill_norms.head()

,Circ,"Standard avarage drilling, m/h","time to well drill, h"
0,Direct,2.7,210
1,RC,5.4,87


In [662]:
df_query_1 = load_from_db_sqlalchemy(DRIVER, SERVER, DATABASE_1, QUERY_1, START_DATE, END_DATE)

In [663]:
df_drilled = load_from_db_sqlalchemy(DRIVER, SERVER, DATABASE_2, QUERY_2, START_DATE, END_DATE)
df_drilled


,HOLEID,DrillCompany,DrillRig,HolePurpose,HoleStatus,ENDDATE,DEPTH
0,MSK47_08_03_A2,BurGeoProekt,ZIF1200_01-BGP,PRODUCT,ACCEPTED,2022-01-24,526.0
1,MSK34_04_20_B_,BurGeoProekt,ZIF1200_06-BGP,PRODUCT,ACCEPTED,2022-01-14,513.0
2,MSK34_06_13_A_,BurGeoProekt,ZIF1200_28-BGP,PRODUCT,ACCEPTED,2022-01-16,515.0
3,MSK34_06_11_A_,BurGeoProekt,ZIF1200_29-BGP,PRODUCT,LIQUID,2022-01-12,514.0
4,MSK46_04_04_B_,BurGeoProekt,ZIF1200_11-BGP,PRODUCT,ACCEPTED,2022-01-12,527.0
...,...,...,...,...,...,...,...
5998,TSU63_08_07_A7,TechnoService-Eng,ZIF1200_02-TSE,PRODUCT,ACCEPTED,2025-10-12,405.0
5999,TSU63_06_02_A7,TechnoService-Eng,ZIF1200_14-TSE,PRODUCT,ACCEPTED,2025-10-21,405.0
6000,TSU63_04_05_A7,TechnoService-Eng,ZIF1200_02-TSE,PRODUCT,ACCEPTED,2025-10-21,402.0
6001,TSU63_04_03_A7,TechnoService-Eng,ZIF1200_14-TSE,PRODUCT,ACCEPTED,2025-10-30,402.0


In [664]:
df_query_1['duration_hours'] = (df_query_1['endtime'] - df_query_1['starttime']).dt.total_seconds() / 3600
df_query_1.head()

,equipment,event,starttime,endtime,duration_hours
0,X481ACD SENNEBOGEN 630-R-HD КРАН,SB_WAITING_FOR_WORK,2022-02-08 20:00:00,2022-03-06 08:00:00,612.0
1,X481ACD SENNEBOGEN 630-R-HD КРАН,SB_WAITING_FOR_WORK,2022-03-08 10:30:00,2022-03-30 08:00:00,525.5
2,ZIF 1200 №16 МК-49,SB_WAITING_FOR_MANPOWER,2022-03-23 09:00:00,2022-04-09 20:00:00,419.0
3,X481ACD SENNEBOGEN 630-R-HD КРАН,SB_WAITING_FOR_WORK,2022-04-15 13:30:00,2022-05-05 08:00:00,474.5
4,ZIF 1200 №8 Бургеопроект,SB_EXPECTED_TOOL,2022-04-23 08:00:00,2022-05-14 08:00:00,504.0


In [665]:
df_merged = df_query_1.merge(
    df_rig_name,
    how='left',
    left_on='equipment',
    right_on='Rig-osiDEM'
)

df_merged['duration_hours'] = df_merged['duration_hours'].clip(upper=24 * 30)
 

df_merged

,equipment,event,starttime,endtime,duration_hours,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation
0,X481ACD SENNEBOGEN 630-R-HD КРАН,SB_WAITING_FOR_WORK,2022-02-08 20:00:00,2022-03-06 08:00:00,612.0,NaN,NaN,NaN,NaN
1,X481ACD SENNEBOGEN 630-R-HD КРАН,SB_WAITING_FOR_WORK,2022-03-08 10:30:00,2022-03-30 08:00:00,525.5,NaN,NaN,NaN,NaN
2,ZIF 1200 №16 МК-49,SB_WAITING_FOR_MANPOWER,2022-03-23 09:00:00,2022-04-09 20:00:00,419.0,Mechkolonna,ZIF1200_16-MK49,ZIF 1200 №16 МК-49,Direct
3,X481ACD SENNEBOGEN 630-R-HD КРАН,SB_WAITING_FOR_WORK,2022-04-15 13:30:00,2022-05-05 08:00:00,474.5,NaN,NaN,NaN,NaN
4,ZIF 1200 №8 Бургеопроект,SB_EXPECTED_TOOL,2022-04-23 08:00:00,2022-05-14 08:00:00,504.0,BurGeoProekt,ZIF1200_08-BGP,ZIF 1200 №8 Бургеопроект,Direct
...,...,...,...,...,...,...,...,...,...
474844,ZIF 1200 №21 МК-49,DR_DRILL_EQUIPEMENT_RECOVERY,2025-10-18 22:00:00,2025-10-20 21:00:00,47.0,Mechkolonna,ZIF1200_21-MK49,ZIF 1200 №21 МК-49,Direct
474845,ZIF 1200 №12 МК-49,SB_ELECTRICAL_SHUTDOWN,2025-10-21 09:00:00,2025-10-22 21:00:00,36.0,Mechkolonna,ZIF1200_12-MK49,ZIF 1200 №12 МК-49,Direct
474846,H757214 URAL 4320 URB 3A3 №4,SB_WAITING_FOR_WORK,2025-10-27 09:00:00,2025-11-01 09:00:00,120.0,NaN,NaN,NaN,NaN
474847,ZIF 1200 №3 BSS,DR_PILOT_DRILLING,2025-10-29 22:00:00,2025-10-31 09:00:00,35.0,BurServisSnab,PRAKLA_03-BSS,ZIF 1200 №3 BSS,Direct


In [666]:
df_merged['duration_hours'].describe()

count    474840.000000
mean          3.574957
std          10.045318
min           0.500000
25%           1.000000
50%           2.000000
75%           4.000000
max         720.000000
Name: duration_hours, dtype: float64

In [667]:
df_final = df_merged.merge(
    df_operations,
    how='left',
    left_on='event',
    right_on='Events'
)
df_final = df_final.drop(columns=['Events'])
df_final = df_final.dropna(subset=['Rig-osiDEM', 'DrillCompany', 'Category of Losses OEE'])


In [668]:
df_final['year_month'] = df_final['starttime'].dt.strftime("%Y-%m")



In [669]:
# 1) Условие для первого дня до 08:00
mask_first_day = (
    (df_final['starttime'].dt.day == 1) &
    (df_final['starttime'].dt.hour < 8)
)

# 2) Условие для последнего дня месяца после 20:00
# Определяем последний день каждого месяца
last_day = df_final['starttime'] + pd.offsets.MonthEnd(0)

mask_last_day = (
    (df_final['starttime'].dt.day == last_day.dt.day) &
    (df_final['starttime'].dt.hour > 20)
)

# Итог: удаляем, если выполняется хотя бы одно условие
df_final = df_final[~(mask_first_day | mask_last_day)]

In [670]:
df_final_2 = df_final.drop(columns=['starttime', 'endtime'])

In [671]:
df_final_2

,equipment,event,duration_hours,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation,Category of Losses OEE,Type of work,year_month
2,ZIF 1200 №16 МК-49,SB_WAITING_FOR_MANPOWER,419.0,Mechkolonna,ZIF1200_16-MK49,ZIF 1200 №16 МК-49,Direct,Unplanned_downtime_losses,Other unplanned Stby time linked with GEOS (wa...,2022-03
4,ZIF 1200 №8 Бургеопроект,SB_EXPECTED_TOOL,504.0,BurGeoProekt,ZIF1200_08-BGP,ZIF 1200 №8 Бургеопроект,Direct,Unplanned_downtime_losses,Other unplanned Stby time linked with DRIL (wa...,2022-04
9,ZIF1200 №16 (TSE),SB_WAITING_FOR_WORK,371.0,TechnoService-Eng,ZIF1200_16-TSE,ZIF1200 №16 (TSE),Direct,Unplanned_downtime_losses,Other unplanned Stby time linked with DRIL (wa...,2022-12
11,ZIF 1200 №11 МК-49,RP_BROKEN_GENERATOR,192.0,Mechkolonna,ZIF1200_11-MK49,ZIF 1200 №11 МК-49,Direct,Unplanned_downtime_losses,Unplanned repairs,2023-01
12,ZIF 1200 №11 МК-49,RP_WAITING_FOR_SPARE_PART,504.0,Mechkolonna,ZIF1200_11-MK49,ZIF 1200 №11 МК-49,Direct,Unplanned_downtime_losses,Other waiting time ( waiting for parts),2023-01
...,...,...,...,...,...,...,...,...,...,...
474827,Prakla №2_ KATCO,RP_WAITING_FOR_SPARE_PART,31.0,KATCO,PRAKLA_02-KAT,Prakla №2_ KATCO,RC,Unplanned_downtime_losses,Other waiting time ( waiting for parts),2025-09
474832,ZIF 1200 №21 МК-49,SB_EXPECTED_TRACTOR,35.0,Mechkolonna,ZIF1200_21-MK49,ZIF 1200 №21 МК-49,Direct,Unplanned_downtime_losses,Other unplanned Stby time linked with DRIL (wa...,2025-10
474833,ZIF 1200 №13 МК-49,RP_BROKEN_GENERATOR,36.0,Mechkolonna,ZIF1200_13-MK49,ZIF 1200 №13 МК-49,Direct,Unplanned_downtime_losses,Unplanned repairs,2025-10
474834,ZIF 1200 №4 МК-49,RP_BROKEN_GENERATOR,96.0,Mechkolonna,ZIF1200_04-MK49,ZIF 1200 №4 МК-49,Direct,Unplanned_downtime_losses,Unplanned repairs,2025-10


In [672]:
df_grouped = (
    df_final
    .groupby(
        [
            'year_month',
            'DrillCompany',
            'equipment',
            'Category of Losses OEE'
        ],
        as_index=False
    )['duration_hours']
    .sum()
)

df_grouped = df_grouped.rename(columns={'duration_hours': 'total_hours'})


In [673]:
df_grouped

,year_month,DrillCompany,equipment,Category of Losses OEE,total_hours
0,2022-01,BurGeoProekt,ZIF 1200 №1 Бургеопроект,Planned_downtime,92.0
1,2022-01,BurGeoProekt,ZIF 1200 №1 Бургеопроект,Unplanned_downtime_losses,42.0
2,2022-01,BurGeoProekt,ZIF 1200 №11 Бургеопроект,Planned_downtime,90.0
3,2022-01,BurGeoProekt,ZIF 1200 №11 Бургеопроект,Unplanned_downtime_losses,48.0
4,2022-01,BurGeoProekt,ZIF 1200 №13 Бургеопроект,Planned_downtime,103.0
...,...,...,...,...,...
4511,2025-10,TechnoService-Eng,ZIF 1200 №5 (TSE),Unplanned_downtime_losses,60.0
4512,2025-10,TechnoService-Eng,ZIF 1200 №6 (TSE),Planned_downtime,81.0
4513,2025-10,TechnoService-Eng,ZIF 1200 №6 (TSE),Unplanned_downtime_losses,79.0
4514,2025-10,TechnoService-Eng,ZIF1200 №14 (TSE),Planned_downtime,95.0


In [674]:
# Pivot таблица
df_pivot = df_grouped.pivot_table(
    index=['year_month', 'DrillCompany', 'equipment'],
    columns='Category of Losses OEE',
    values='total_hours',
    aggfunc='sum',
    fill_value=0
).reset_index()

df_pivot.columns.name = None

# Добавляем только доступные часы
df_pivot['available_hours'] = (
    pd.to_datetime(df_pivot['year_month'] + "-01")  # 1-е число месяца
    .apply(lambda d: (d + pd.offsets.MonthEnd(0)).day)  # число дней в месяце
    * 24  # перевод в часы
)


In [675]:
df_pivot

,year_month,DrillCompany,equipment,Planned_downtime,Unplanned_downtime_losses,available_hours
0,2022-01,BurGeoProekt,ZIF 1200 №1 Бургеопроект,92.0,42.0,744
1,2022-01,BurGeoProekt,ZIF 1200 №11 Бургеопроект,90.0,48.0,744
2,2022-01,BurGeoProekt,ZIF 1200 №13 Бургеопроект,103.0,97.0,744
3,2022-01,BurGeoProekt,ZIF 1200 №14 Бургеопроект,87.0,43.0,744
4,2022-01,BurGeoProekt,ZIF 1200 №2 Бургеопроект,83.0,57.0,744
...,...,...,...,...,...,...
2272,2025-10,TechnoService-Eng,ZIF 1200 №2 (TSE),95.0,90.0,744
2273,2025-10,TechnoService-Eng,ZIF 1200 №4 (TSE),2.0,5.0,744
2274,2025-10,TechnoService-Eng,ZIF 1200 №5 (TSE),92.0,60.0,744
2275,2025-10,TechnoService-Eng,ZIF 1200 №6 (TSE),81.0,79.0,744


In [676]:
df_pivot['planned_factor'] = 1 - df_pivot['Planned_downtime'] / df_pivot['available_hours']

# Заменяем ошибки и NaN → 0
df_pivot['planned_factor'] = df_pivot['planned_factor'].fillna(0)

# Ограничение диапазона 0…1
df_pivot['planned_factor'] = df_pivot['planned_factor'].clip(lower=0, upper=1)

# 4. Availability: 1 – (Unplanned / (available – Planned))
df_pivot['availability'] = 1 - (
    df_pivot['Unplanned_downtime_losses'] /
    (df_pivot['available_hours'] - df_pivot['Planned_downtime'])
)

df_pivot['availability'] = df_pivot['availability'].fillna(0).clip(0, 1)

In [677]:
df_pivot = df_pivot.merge(
    df_rig_name[['Rig-osiDEM', 'Tipe of circulation']],
    left_on='equipment',
    right_on='Rig-osiDEM',
    how='left'
)
df_pivot

,year_month,DrillCompany,equipment,Planned_downtime,Unplanned_downtime_losses,available_hours,planned_factor,availability,Rig-osiDEM,Tipe of circulation
0,2022-01,BurGeoProekt,ZIF 1200 №1 Бургеопроект,92.0,42.0,744,0.876344,0.935583,ZIF 1200 №1 Бургеопроект,Direct
1,2022-01,BurGeoProekt,ZIF 1200 №11 Бургеопроект,90.0,48.0,744,0.879032,0.926606,ZIF 1200 №11 Бургеопроект,Direct
2,2022-01,BurGeoProekt,ZIF 1200 №13 Бургеопроект,103.0,97.0,744,0.861559,0.848674,ZIF 1200 №13 Бургеопроект,Direct
3,2022-01,BurGeoProekt,ZIF 1200 №14 Бургеопроект,87.0,43.0,744,0.883065,0.934551,ZIF 1200 №14 Бургеопроект,Direct
4,2022-01,BurGeoProekt,ZIF 1200 №2 Бургеопроект,83.0,57.0,744,0.888441,0.913767,ZIF 1200 №2 Бургеопроект,Direct
...,...,...,...,...,...,...,...,...,...,...
2283,2025-10,TechnoService-Eng,ZIF 1200 №2 (TSE),95.0,90.0,744,0.872312,0.861325,ZIF 1200 №2 (TSE),Direct
2284,2025-10,TechnoService-Eng,ZIF 1200 №4 (TSE),2.0,5.0,744,0.997312,0.993261,ZIF 1200 №4 (TSE),Direct
2285,2025-10,TechnoService-Eng,ZIF 1200 №5 (TSE),92.0,60.0,744,0.876344,0.907975,ZIF 1200 №5 (TSE),Direct
2286,2025-10,TechnoService-Eng,ZIF 1200 №6 (TSE),81.0,79.0,744,0.891129,0.880845,ZIF 1200 №6 (TSE),Direct


присоеденить коэфициент из df_drill_norms умноженный на доступное время

In [678]:
from regex import T


df_pivot = df_pivot.merge(
    df_drill_norms[['Circ', 'Standard avarage drilling, m/h', 'time to well drill, h']],
    left_on='Tipe of circulation',
    right_on='Circ',
    how='left'
).drop(columns=['equipment', 'Tipe of circulation'])

df_pivot



,year_month,DrillCompany,Planned_downtime,Unplanned_downtime_losses,available_hours,planned_factor,availability,Rig-osiDEM,Circ,"Standard avarage drilling, m/h","time to well drill, h"
0,2022-01,BurGeoProekt,92.0,42.0,744,0.876344,0.935583,ZIF 1200 №1 Бургеопроект,Direct,2.7,210
1,2022-01,BurGeoProekt,90.0,48.0,744,0.879032,0.926606,ZIF 1200 №11 Бургеопроект,Direct,2.7,210
2,2022-01,BurGeoProekt,103.0,97.0,744,0.861559,0.848674,ZIF 1200 №13 Бургеопроект,Direct,2.7,210
3,2022-01,BurGeoProekt,87.0,43.0,744,0.883065,0.934551,ZIF 1200 №14 Бургеопроект,Direct,2.7,210
4,2022-01,BurGeoProekt,83.0,57.0,744,0.888441,0.913767,ZIF 1200 №2 Бургеопроект,Direct,2.7,210
...,...,...,...,...,...,...,...,...,...,...,...
2283,2025-10,TechnoService-Eng,95.0,90.0,744,0.872312,0.861325,ZIF 1200 №2 (TSE),Direct,2.7,210
2284,2025-10,TechnoService-Eng,2.0,5.0,744,0.997312,0.993261,ZIF 1200 №4 (TSE),Direct,2.7,210
2285,2025-10,TechnoService-Eng,92.0,60.0,744,0.876344,0.907975,ZIF 1200 №5 (TSE),Direct,2.7,210
2286,2025-10,TechnoService-Eng,81.0,79.0,744,0.891129,0.880845,ZIF 1200 №6 (TSE),Direct,2.7,210


In [679]:
df_pivot['Possible_drilling'] = (df_pivot['available_hours'] - df_pivot['Planned_downtime'] - df_pivot['Unplanned_downtime_losses']) * df_pivot['Standard avarage drilling, m/h']
df_pivot.drop(columns=['Standard avarage drilling, m/h'])

,year_month,DrillCompany,Planned_downtime,Unplanned_downtime_losses,available_hours,planned_factor,availability,Rig-osiDEM,Circ,"time to well drill, h",Possible_drilling
0,2022-01,BurGeoProekt,92.0,42.0,744,0.876344,0.935583,ZIF 1200 №1 Бургеопроект,Direct,210,1647.0
1,2022-01,BurGeoProekt,90.0,48.0,744,0.879032,0.926606,ZIF 1200 №11 Бургеопроект,Direct,210,1636.2
2,2022-01,BurGeoProekt,103.0,97.0,744,0.861559,0.848674,ZIF 1200 №13 Бургеопроект,Direct,210,1468.8
3,2022-01,BurGeoProekt,87.0,43.0,744,0.883065,0.934551,ZIF 1200 №14 Бургеопроект,Direct,210,1657.8
4,2022-01,BurGeoProekt,83.0,57.0,744,0.888441,0.913767,ZIF 1200 №2 Бургеопроект,Direct,210,1630.8
...,...,...,...,...,...,...,...,...,...,...,...
2283,2025-10,TechnoService-Eng,95.0,90.0,744,0.872312,0.861325,ZIF 1200 №2 (TSE),Direct,210,1509.3
2284,2025-10,TechnoService-Eng,2.0,5.0,744,0.997312,0.993261,ZIF 1200 №4 (TSE),Direct,210,1989.9
2285,2025-10,TechnoService-Eng,92.0,60.0,744,0.876344,0.907975,ZIF 1200 №5 (TSE),Direct,210,1598.4
2286,2025-10,TechnoService-Eng,81.0,79.0,744,0.891129,0.880845,ZIF 1200 №6 (TSE),Direct,210,1576.8


In [680]:
df_drilled['year_month'] = df_drilled['ENDDATE'].dt.strftime("%Y-%m")
df_drilled


,HOLEID,DrillCompany,DrillRig,HolePurpose,HoleStatus,ENDDATE,DEPTH,year_month
0,MSK47_08_03_A2,BurGeoProekt,ZIF1200_01-BGP,PRODUCT,ACCEPTED,2022-01-24,526.0,2022-01
1,MSK34_04_20_B_,BurGeoProekt,ZIF1200_06-BGP,PRODUCT,ACCEPTED,2022-01-14,513.0,2022-01
2,MSK34_06_13_A_,BurGeoProekt,ZIF1200_28-BGP,PRODUCT,ACCEPTED,2022-01-16,515.0,2022-01
3,MSK34_06_11_A_,BurGeoProekt,ZIF1200_29-BGP,PRODUCT,LIQUID,2022-01-12,514.0,2022-01
4,MSK46_04_04_B_,BurGeoProekt,ZIF1200_11-BGP,PRODUCT,ACCEPTED,2022-01-12,527.0,2022-01
...,...,...,...,...,...,...,...,...
5998,TSU63_08_07_A7,TechnoService-Eng,ZIF1200_02-TSE,PRODUCT,ACCEPTED,2025-10-12,405.0,2025-10
5999,TSU63_06_02_A7,TechnoService-Eng,ZIF1200_14-TSE,PRODUCT,ACCEPTED,2025-10-21,405.0,2025-10
6000,TSU63_04_05_A7,TechnoService-Eng,ZIF1200_02-TSE,PRODUCT,ACCEPTED,2025-10-21,402.0,2025-10
6001,TSU63_04_03_A7,TechnoService-Eng,ZIF1200_14-TSE,PRODUCT,ACCEPTED,2025-10-30,402.0,2025-10


In [681]:
df_depth_pivot = (
    df_drilled
    .groupby(['year_month', 'DrillCompany', 'DrillRig'])
    .agg(
        total_depth=('DEPTH', 'sum'),
        wells_liquid=('HoleStatus', lambda x: (x == 'LIQUID').sum())
    )
    .reset_index()
)


df_depth_pivot

,year_month,DrillCompany,DrillRig,total_depth,wells_liquid
0,2022-01,BurGeoProekt,ZIF1200_01-BGP,1537.0,0
1,2022-01,BurGeoProekt,ZIF1200_02-BGP,521.0,0
2,2022-01,BurGeoProekt,ZIF1200_03-BGP,502.0,0
3,2022-01,BurGeoProekt,ZIF1200_04-BGP,1027.0,0
4,2022-01,BurGeoProekt,ZIF1200_06-BGP,1053.0,0
...,...,...,...,...,...
2020,2025-10,KATCO,ZMO1500-4-KAT,460.0,0
2021,2025-10,KATCO,ZMO1500-5-KAT,850.0,0
2022,2025-10,KATCO,ZMO1500-6-KAT,1227.0,0
2023,2025-10,TechnoService-Eng,ZIF1200_02-TSE,1604.0,1


In [682]:
df_depth_pivot = df_depth_pivot.merge(
    df_rig_name[['Rig-AcQuire', 'Rig-osiDEM']],
    left_on='DrillRig',
    right_on='Rig-AcQuire',
    how='left'
)



df_depth_pivot

,year_month,DrillCompany,DrillRig,total_depth,wells_liquid,Rig-AcQuire,Rig-osiDEM
0,2022-01,BurGeoProekt,ZIF1200_01-BGP,1537.0,0,ZIF1200_01-BGP,ZIF 1200 №1 Бургеопроект
1,2022-01,BurGeoProekt,ZIF1200_02-BGP,521.0,0,ZIF1200_02-BGP,ZIF 1200 №2 Бургеопроект
2,2022-01,BurGeoProekt,ZIF1200_03-BGP,502.0,0,ZIF1200_03-BGP,ZIF 1200 №3 Бургеопроект
3,2022-01,BurGeoProekt,ZIF1200_04-BGP,1027.0,0,ZIF1200_04-BGP,ZIF 1200 №4 Бургеопроект
4,2022-01,BurGeoProekt,ZIF1200_06-BGP,1053.0,0,ZIF1200_06-BGP,ZIF 1200 №6 Бургеопроект
...,...,...,...,...,...,...,...
2039,2025-10,KATCO,ZMO1500-4-KAT,460.0,0,ZMO1500-4-KAT,ПБУ ЗМО-1500ПС №4
2040,2025-10,KATCO,ZMO1500-5-KAT,850.0,0,ZMO1500-5-KAT,ПБУ ЗМО-1500ПС №5
2041,2025-10,KATCO,ZMO1500-6-KAT,1227.0,0,ZMO1500-6-KAT,ПБУ ЗМО-1500ПС №6
2042,2025-10,TechnoService-Eng,ZIF1200_02-TSE,1604.0,1,ZIF1200_02-TSE,ZIF 1200 №2 (TSE)


In [683]:
df_final_3 = df_pivot.merge(
    df_depth_pivot,
    left_on=['year_month', 'Rig-osiDEM'],
    right_on=['year_month', 'Rig-osiDEM'],
    how='left',
    suffixes=('', '_depth')  # чтобы НЕ было _x/_y
)


df_final_3


,year_month,DrillCompany,Planned_downtime,Unplanned_downtime_losses,available_hours,planned_factor,availability,Rig-osiDEM,Circ,"Standard avarage drilling, m/h","time to well drill, h",Possible_drilling,DrillCompany_depth,DrillRig,total_depth,wells_liquid,Rig-AcQuire
0,2022-01,BurGeoProekt,92.0,42.0,744,0.876344,0.935583,ZIF 1200 №1 Бургеопроект,Direct,2.7,210,1647.0,BurGeoProekt,ZIF1200_01-BGP,1537.0,0.0,ZIF1200_01-BGP
1,2022-01,BurGeoProekt,90.0,48.0,744,0.879032,0.926606,ZIF 1200 №11 Бургеопроект,Direct,2.7,210,1636.2,BurGeoProekt,ZIF1200_11-BGP,1045.0,0.0,ZIF1200_11-BGP
2,2022-01,BurGeoProekt,103.0,97.0,744,0.861559,0.848674,ZIF 1200 №13 Бургеопроект,Direct,2.7,210,1468.8,BurGeoProekt,ZIF1200_13-BGP,1005.0,0.0,ZIF1200_13-BGP
3,2022-01,BurGeoProekt,87.0,43.0,744,0.883065,0.934551,ZIF 1200 №14 Бургеопроект,Direct,2.7,210,1657.8,BurGeoProekt,ZIF1200_14-BGP,1012.0,0.0,ZIF1200_14-BGP
4,2022-01,BurGeoProekt,83.0,57.0,744,0.888441,0.913767,ZIF 1200 №2 Бургеопроект,Direct,2.7,210,1630.8,BurGeoProekt,ZIF1200_02-BGP,521.0,0.0,ZIF1200_02-BGP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,2025-10,TechnoService-Eng,95.0,90.0,744,0.872312,0.861325,ZIF 1200 №2 (TSE),Direct,2.7,210,1509.3,TechnoService-Eng,ZIF1200_02-TSE,1604.0,1.0,ZIF1200_02-TSE
2288,2025-10,TechnoService-Eng,2.0,5.0,744,0.997312,0.993261,ZIF 1200 №4 (TSE),Direct,2.7,210,1989.9,NaN,NaN,NaN,NaN,NaN
2289,2025-10,TechnoService-Eng,92.0,60.0,744,0.876344,0.907975,ZIF 1200 №5 (TSE),Direct,2.7,210,1598.4,NaN,NaN,NaN,NaN,NaN
2290,2025-10,TechnoService-Eng,81.0,79.0,744,0.891129,0.880845,ZIF 1200 №6 (TSE),Direct,2.7,210,1576.8,NaN,NaN,NaN,NaN,NaN


In [684]:
df_final3 = df_final_3.drop(columns=[
    'DrillRig',
    'Rig-AcQuire',
    'Rig-AcQuire_depth',
    'Rig-osiDEM_depth',
    'DrillCompany_depth'
], errors='ignore')   # ignore — чтобы не ругался, если какой-то нет
df_final_3

,year_month,DrillCompany,Planned_downtime,Unplanned_downtime_losses,available_hours,planned_factor,availability,Rig-osiDEM,Circ,"Standard avarage drilling, m/h","time to well drill, h",Possible_drilling,DrillCompany_depth,DrillRig,total_depth,wells_liquid,Rig-AcQuire
0,2022-01,BurGeoProekt,92.0,42.0,744,0.876344,0.935583,ZIF 1200 №1 Бургеопроект,Direct,2.7,210,1647.0,BurGeoProekt,ZIF1200_01-BGP,1537.0,0.0,ZIF1200_01-BGP
1,2022-01,BurGeoProekt,90.0,48.0,744,0.879032,0.926606,ZIF 1200 №11 Бургеопроект,Direct,2.7,210,1636.2,BurGeoProekt,ZIF1200_11-BGP,1045.0,0.0,ZIF1200_11-BGP
2,2022-01,BurGeoProekt,103.0,97.0,744,0.861559,0.848674,ZIF 1200 №13 Бургеопроект,Direct,2.7,210,1468.8,BurGeoProekt,ZIF1200_13-BGP,1005.0,0.0,ZIF1200_13-BGP
3,2022-01,BurGeoProekt,87.0,43.0,744,0.883065,0.934551,ZIF 1200 №14 Бургеопроект,Direct,2.7,210,1657.8,BurGeoProekt,ZIF1200_14-BGP,1012.0,0.0,ZIF1200_14-BGP
4,2022-01,BurGeoProekt,83.0,57.0,744,0.888441,0.913767,ZIF 1200 №2 Бургеопроект,Direct,2.7,210,1630.8,BurGeoProekt,ZIF1200_02-BGP,521.0,0.0,ZIF1200_02-BGP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,2025-10,TechnoService-Eng,95.0,90.0,744,0.872312,0.861325,ZIF 1200 №2 (TSE),Direct,2.7,210,1509.3,TechnoService-Eng,ZIF1200_02-TSE,1604.0,1.0,ZIF1200_02-TSE
2288,2025-10,TechnoService-Eng,2.0,5.0,744,0.997312,0.993261,ZIF 1200 №4 (TSE),Direct,2.7,210,1989.9,NaN,NaN,NaN,NaN,NaN
2289,2025-10,TechnoService-Eng,92.0,60.0,744,0.876344,0.907975,ZIF 1200 №5 (TSE),Direct,2.7,210,1598.4,NaN,NaN,NaN,NaN,NaN
2290,2025-10,TechnoService-Eng,81.0,79.0,744,0.891129,0.880845,ZIF 1200 №6 (TSE),Direct,2.7,210,1576.8,NaN,NaN,NaN,NaN,NaN


In [685]:

df_final_3[['total_depth', 'wells_liquid']] = (
    df_final_3[['total_depth', 'wells_liquid']].fillna(0)
)


# сначала переименуем колонку, если там есть скрытый TAB
# df_final_2 = df_final_2.rename(columns=lambda x: x.strip())

# безопасное вычисление Performance
df_final_3['Performance'] = df_final_3.apply(
    lambda row: row['total_depth'] / row['Possible_drilling']
    if row['Possible_drilling'] not in [0, None, float('nan')]
    else 0,
    axis=1
)

In [686]:
df_final_3

,year_month,DrillCompany,Planned_downtime,Unplanned_downtime_losses,available_hours,planned_factor,availability,Rig-osiDEM,Circ,"Standard avarage drilling, m/h","time to well drill, h",Possible_drilling,DrillCompany_depth,DrillRig,total_depth,wells_liquid,Rig-AcQuire,Performance
0,2022-01,BurGeoProekt,92.0,42.0,744,0.876344,0.935583,ZIF 1200 №1 Бургеопроект,Direct,2.7,210,1647.0,BurGeoProekt,ZIF1200_01-BGP,1537.0,0.0,ZIF1200_01-BGP,0.933212
1,2022-01,BurGeoProekt,90.0,48.0,744,0.879032,0.926606,ZIF 1200 №11 Бургеопроект,Direct,2.7,210,1636.2,BurGeoProekt,ZIF1200_11-BGP,1045.0,0.0,ZIF1200_11-BGP,0.638675
2,2022-01,BurGeoProekt,103.0,97.0,744,0.861559,0.848674,ZIF 1200 №13 Бургеопроект,Direct,2.7,210,1468.8,BurGeoProekt,ZIF1200_13-BGP,1005.0,0.0,ZIF1200_13-BGP,0.684232
3,2022-01,BurGeoProekt,87.0,43.0,744,0.883065,0.934551,ZIF 1200 №14 Бургеопроект,Direct,2.7,210,1657.8,BurGeoProekt,ZIF1200_14-BGP,1012.0,0.0,ZIF1200_14-BGP,0.610448
4,2022-01,BurGeoProekt,83.0,57.0,744,0.888441,0.913767,ZIF 1200 №2 Бургеопроект,Direct,2.7,210,1630.8,BurGeoProekt,ZIF1200_02-BGP,521.0,0.0,ZIF1200_02-BGP,0.319475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,2025-10,TechnoService-Eng,95.0,90.0,744,0.872312,0.861325,ZIF 1200 №2 (TSE),Direct,2.7,210,1509.3,TechnoService-Eng,ZIF1200_02-TSE,1604.0,1.0,ZIF1200_02-TSE,1.062744
2288,2025-10,TechnoService-Eng,2.0,5.0,744,0.997312,0.993261,ZIF 1200 №4 (TSE),Direct,2.7,210,1989.9,NaN,NaN,0.0,0.0,NaN,0.000000
2289,2025-10,TechnoService-Eng,92.0,60.0,744,0.876344,0.907975,ZIF 1200 №5 (TSE),Direct,2.7,210,1598.4,NaN,NaN,0.0,0.0,NaN,0.000000
2290,2025-10,TechnoService-Eng,81.0,79.0,744,0.891129,0.880845,ZIF 1200 №6 (TSE),Direct,2.7,210,1576.8,NaN,NaN,0.0,0.0,NaN,0.000000


In [687]:
df_final_3['Planned prduction time'] = df_final_3['available_hours'] - df_final_3['Planned_downtime']
df_final_3['Gross operaiting time'] = df_final_3['Planned prduction time'] - df_final_3['Unplanned_downtime_losses']
# Расчёт по условию
df_final_3['Net operating time'] = df_final_3.apply(
    lambda row: row['Gross operaiting time']
    if row['Performance'] > 1
    else row['Gross operaiting time'] * row['Performance'],
    axis=1
)


In [688]:
df_final_3

,year_month,DrillCompany,Planned_downtime,Unplanned_downtime_losses,available_hours,planned_factor,availability,Rig-osiDEM,Circ,"Standard avarage drilling, m/h",...,Possible_drilling,DrillCompany_depth,DrillRig,total_depth,wells_liquid,Rig-AcQuire,Performance,Planned prduction time,Gross operaiting time,Net operating time
0,2022-01,BurGeoProekt,92.0,42.0,744,0.876344,0.935583,ZIF 1200 №1 Бургеопроект,Direct,2.7,...,1647.0,BurGeoProekt,ZIF1200_01-BGP,1537.0,0.0,ZIF1200_01-BGP,0.933212,652.0,610.0,569.259259
1,2022-01,BurGeoProekt,90.0,48.0,744,0.879032,0.926606,ZIF 1200 №11 Бургеопроект,Direct,2.7,...,1636.2,BurGeoProekt,ZIF1200_11-BGP,1045.0,0.0,ZIF1200_11-BGP,0.638675,654.0,606.0,387.037037
2,2022-01,BurGeoProekt,103.0,97.0,744,0.861559,0.848674,ZIF 1200 №13 Бургеопроект,Direct,2.7,...,1468.8,BurGeoProekt,ZIF1200_13-BGP,1005.0,0.0,ZIF1200_13-BGP,0.684232,641.0,544.0,372.222222
3,2022-01,BurGeoProekt,87.0,43.0,744,0.883065,0.934551,ZIF 1200 №14 Бургеопроект,Direct,2.7,...,1657.8,BurGeoProekt,ZIF1200_14-BGP,1012.0,0.0,ZIF1200_14-BGP,0.610448,657.0,614.0,374.814815
4,2022-01,BurGeoProekt,83.0,57.0,744,0.888441,0.913767,ZIF 1200 №2 Бургеопроект,Direct,2.7,...,1630.8,BurGeoProekt,ZIF1200_02-BGP,521.0,0.0,ZIF1200_02-BGP,0.319475,661.0,604.0,192.962963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,2025-10,TechnoService-Eng,95.0,90.0,744,0.872312,0.861325,ZIF 1200 №2 (TSE),Direct,2.7,...,1509.3,TechnoService-Eng,ZIF1200_02-TSE,1604.0,1.0,ZIF1200_02-TSE,1.062744,649.0,559.0,559.000000
2288,2025-10,TechnoService-Eng,2.0,5.0,744,0.997312,0.993261,ZIF 1200 №4 (TSE),Direct,2.7,...,1989.9,NaN,NaN,0.0,0.0,NaN,0.000000,742.0,737.0,0.000000
2289,2025-10,TechnoService-Eng,92.0,60.0,744,0.876344,0.907975,ZIF 1200 №5 (TSE),Direct,2.7,...,1598.4,NaN,NaN,0.0,0.0,NaN,0.000000,652.0,592.0,0.000000
2290,2025-10,TechnoService-Eng,81.0,79.0,744,0.891129,0.880845,ZIF 1200 №6 (TSE),Direct,2.7,...,1576.8,NaN,NaN,0.0,0.0,NaN,0.000000,663.0,584.0,0.000000


In [689]:
df_final_3['Valuable operating time'] = df_final_3['Net operating time'] - df_final_3['wells_liquid'] * df_final_3['time to well drill, h']

In [690]:
df_final_3

,year_month,DrillCompany,Planned_downtime,Unplanned_downtime_losses,available_hours,planned_factor,availability,Rig-osiDEM,Circ,"Standard avarage drilling, m/h",...,DrillCompany_depth,DrillRig,total_depth,wells_liquid,Rig-AcQuire,Performance,Planned prduction time,Gross operaiting time,Net operating time,Valuable operating time
0,2022-01,BurGeoProekt,92.0,42.0,744,0.876344,0.935583,ZIF 1200 №1 Бургеопроект,Direct,2.7,...,BurGeoProekt,ZIF1200_01-BGP,1537.0,0.0,ZIF1200_01-BGP,0.933212,652.0,610.0,569.259259,569.259259
1,2022-01,BurGeoProekt,90.0,48.0,744,0.879032,0.926606,ZIF 1200 №11 Бургеопроект,Direct,2.7,...,BurGeoProekt,ZIF1200_11-BGP,1045.0,0.0,ZIF1200_11-BGP,0.638675,654.0,606.0,387.037037,387.037037
2,2022-01,BurGeoProekt,103.0,97.0,744,0.861559,0.848674,ZIF 1200 №13 Бургеопроект,Direct,2.7,...,BurGeoProekt,ZIF1200_13-BGP,1005.0,0.0,ZIF1200_13-BGP,0.684232,641.0,544.0,372.222222,372.222222
3,2022-01,BurGeoProekt,87.0,43.0,744,0.883065,0.934551,ZIF 1200 №14 Бургеопроект,Direct,2.7,...,BurGeoProekt,ZIF1200_14-BGP,1012.0,0.0,ZIF1200_14-BGP,0.610448,657.0,614.0,374.814815,374.814815
4,2022-01,BurGeoProekt,83.0,57.0,744,0.888441,0.913767,ZIF 1200 №2 Бургеопроект,Direct,2.7,...,BurGeoProekt,ZIF1200_02-BGP,521.0,0.0,ZIF1200_02-BGP,0.319475,661.0,604.0,192.962963,192.962963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,2025-10,TechnoService-Eng,95.0,90.0,744,0.872312,0.861325,ZIF 1200 №2 (TSE),Direct,2.7,...,TechnoService-Eng,ZIF1200_02-TSE,1604.0,1.0,ZIF1200_02-TSE,1.062744,649.0,559.0,559.000000,349.000000
2288,2025-10,TechnoService-Eng,2.0,5.0,744,0.997312,0.993261,ZIF 1200 №4 (TSE),Direct,2.7,...,NaN,NaN,0.0,0.0,NaN,0.000000,742.0,737.0,0.000000,0.000000
2289,2025-10,TechnoService-Eng,92.0,60.0,744,0.876344,0.907975,ZIF 1200 №5 (TSE),Direct,2.7,...,NaN,NaN,0.0,0.0,NaN,0.000000,652.0,592.0,0.000000,0.000000
2290,2025-10,TechnoService-Eng,81.0,79.0,744,0.891129,0.880845,ZIF 1200 №6 (TSE),Direct,2.7,...,NaN,NaN,0.0,0.0,NaN,0.000000,663.0,584.0,0.000000,0.000000


In [691]:
df_final_3 = df_final_3.drop(columns=[
    'Circ',
    'Standard avarage drilling, m/h',
    'time to well drill, h',
    'Possible_drilling',
    'total_depth',
    'wells_liquid',
])


In [692]:
df_final_3

,year_month,DrillCompany,Planned_downtime,Unplanned_downtime_losses,available_hours,planned_factor,availability,Rig-osiDEM,DrillCompany_depth,DrillRig,Rig-AcQuire,Performance,Planned prduction time,Gross operaiting time,Net operating time,Valuable operating time
0,2022-01,BurGeoProekt,92.0,42.0,744,0.876344,0.935583,ZIF 1200 №1 Бургеопроект,BurGeoProekt,ZIF1200_01-BGP,ZIF1200_01-BGP,0.933212,652.0,610.0,569.259259,569.259259
1,2022-01,BurGeoProekt,90.0,48.0,744,0.879032,0.926606,ZIF 1200 №11 Бургеопроект,BurGeoProekt,ZIF1200_11-BGP,ZIF1200_11-BGP,0.638675,654.0,606.0,387.037037,387.037037
2,2022-01,BurGeoProekt,103.0,97.0,744,0.861559,0.848674,ZIF 1200 №13 Бургеопроект,BurGeoProekt,ZIF1200_13-BGP,ZIF1200_13-BGP,0.684232,641.0,544.0,372.222222,372.222222
3,2022-01,BurGeoProekt,87.0,43.0,744,0.883065,0.934551,ZIF 1200 №14 Бургеопроект,BurGeoProekt,ZIF1200_14-BGP,ZIF1200_14-BGP,0.610448,657.0,614.0,374.814815,374.814815
4,2022-01,BurGeoProekt,83.0,57.0,744,0.888441,0.913767,ZIF 1200 №2 Бургеопроект,BurGeoProekt,ZIF1200_02-BGP,ZIF1200_02-BGP,0.319475,661.0,604.0,192.962963,192.962963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,2025-10,TechnoService-Eng,95.0,90.0,744,0.872312,0.861325,ZIF 1200 №2 (TSE),TechnoService-Eng,ZIF1200_02-TSE,ZIF1200_02-TSE,1.062744,649.0,559.0,559.000000,349.000000
2288,2025-10,TechnoService-Eng,2.0,5.0,744,0.997312,0.993261,ZIF 1200 №4 (TSE),NaN,NaN,NaN,0.000000,742.0,737.0,0.000000,0.000000
2289,2025-10,TechnoService-Eng,92.0,60.0,744,0.876344,0.907975,ZIF 1200 №5 (TSE),NaN,NaN,NaN,0.000000,652.0,592.0,0.000000,0.000000
2290,2025-10,TechnoService-Eng,81.0,79.0,744,0.891129,0.880845,ZIF 1200 №6 (TSE),NaN,NaN,NaN,0.000000,663.0,584.0,0.000000,0.000000


In [693]:
df_final_3['Quality'] = df_final_3.apply(
    lambda row: row['Valuable operating time'] / row['Net operating time']
    if row['Net operating time'] not in [0, None, float('nan')]
    else 0,
    axis=1
)

df_final_3

,year_month,DrillCompany,Planned_downtime,Unplanned_downtime_losses,available_hours,planned_factor,availability,Rig-osiDEM,DrillCompany_depth,DrillRig,Rig-AcQuire,Performance,Planned prduction time,Gross operaiting time,Net operating time,Valuable operating time,Quality
0,2022-01,BurGeoProekt,92.0,42.0,744,0.876344,0.935583,ZIF 1200 №1 Бургеопроект,BurGeoProekt,ZIF1200_01-BGP,ZIF1200_01-BGP,0.933212,652.0,610.0,569.259259,569.259259,1.000000
1,2022-01,BurGeoProekt,90.0,48.0,744,0.879032,0.926606,ZIF 1200 №11 Бургеопроект,BurGeoProekt,ZIF1200_11-BGP,ZIF1200_11-BGP,0.638675,654.0,606.0,387.037037,387.037037,1.000000
2,2022-01,BurGeoProekt,103.0,97.0,744,0.861559,0.848674,ZIF 1200 №13 Бургеопроект,BurGeoProekt,ZIF1200_13-BGP,ZIF1200_13-BGP,0.684232,641.0,544.0,372.222222,372.222222,1.000000
3,2022-01,BurGeoProekt,87.0,43.0,744,0.883065,0.934551,ZIF 1200 №14 Бургеопроект,BurGeoProekt,ZIF1200_14-BGP,ZIF1200_14-BGP,0.610448,657.0,614.0,374.814815,374.814815,1.000000
4,2022-01,BurGeoProekt,83.0,57.0,744,0.888441,0.913767,ZIF 1200 №2 Бургеопроект,BurGeoProekt,ZIF1200_02-BGP,ZIF1200_02-BGP,0.319475,661.0,604.0,192.962963,192.962963,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,2025-10,TechnoService-Eng,95.0,90.0,744,0.872312,0.861325,ZIF 1200 №2 (TSE),TechnoService-Eng,ZIF1200_02-TSE,ZIF1200_02-TSE,1.062744,649.0,559.0,559.000000,349.000000,0.624329
2288,2025-10,TechnoService-Eng,2.0,5.0,744,0.997312,0.993261,ZIF 1200 №4 (TSE),NaN,NaN,NaN,0.000000,742.0,737.0,0.000000,0.000000,0.000000
2289,2025-10,TechnoService-Eng,92.0,60.0,744,0.876344,0.907975,ZIF 1200 №5 (TSE),NaN,NaN,NaN,0.000000,652.0,592.0,0.000000,0.000000,0.000000
2290,2025-10,TechnoService-Eng,81.0,79.0,744,0.891129,0.880845,ZIF 1200 №6 (TSE),NaN,NaN,NaN,0.000000,663.0,584.0,0.000000,0.000000,0.000000


In [694]:
list_of_export_df = ('df_final_2', 'df_depth_pivot', 'df_final_3' )